# Øvelse 4 - Digital Nomads

I denne øvelse kommer vi til, at arbejde med datamanipulation, merging af DataFrames og visualiseringer af forskellige variable. Vi kommer til primært til at kigge på hvordan man kan sikre sig, at ens data har rimelige værdier, hvordan man kan berige sin data ved, at slå flere DataFrames sammen, og hvordan man kan bruge dette til at skabe nogle mere sigende visualiseringer.

In [ ]:
# Vi importerer de nødvendige libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Vi importerer vores data
trips_data = pd.read_csv('https://sds-aau.github.io/SDS-master/M1/data/trips.csv')
people_data = pd.read_csv('https://sds-aau.github.io/SDS-master/M1/data/people.csv')
country_data = pd.read_csv('https://sds-aau.github.io/SDS-master/M1/data/countrylist.csv')

In [ ]:
# Vi konverterer det importerede data til DataFrames
trips = pd.DataFrame(trips_data)
people = pd.DataFrame(people_data)
country = pd.DataFrame(country_data)

## 1 - Datamanipulation og Overblik

In [ ]:
# 1.1: Hvordan ser de første 5 rækker af trips dataframen ud og hvilke kolonner arbejder vi med?


In [ ]:
# 1.2: Er der nogle rækker med manglende data i trips dataframen?
# 1.3: Hvilke datatyper arbejder vi med?


Som det kan ses har vi to kolonner i vores `trips` DataFrame som er af datatypen `object`. Vi vil gerne have at datoer i vores DataFrames er formatteret som `datetime`. Her kan vi bruge funnktionen `pd.to_datetime()` til at konvertere vores kolonner til datatypen `datetime`. Her er vi nødt til, at bruge parametret `errors='coerce'` for, at sikre at fejl bliver håndteret korrekt.

In [ ]:
# 1.4: Konverter kolonnerne date_end og date_start til datatypen datetime.
trips['date_start'] = pd.to_datetime(___, errors=___)
trips['date_end'] = pd.to_datetime(___, errors=___)

In [ ]:
# 1.5: Udregn længden af rejserne i kolonnen trip_duration:
# Dette kan gøres ved at trække slut tidspunktet fra startstidspunktet. 
trips['trip_duration'] = (___ - ___) / np.timedelta64(1, 'D')

In [ ]:
# 1.6: Hvordan ser vores nye kolonne ud, hvad er max og hvad er min trip duration?
print(___)
print(___)

Det ser ud til, at vi har nogle start og slutdatoer der ligger udenfor det der ville være forventeligt. Vi kan kigge nærmere på det vha. et boxplot. Brug funktionen `sns.boxplot()` til at lave et boxplot af `trip_duration`.

In [ ]:
# 1.7: Lav et boxplot af trip_duration for at tjekke for outliers:


Som det kan ses ser det helt underligt ud med mængden og størrelsen af de outliers der er. Vi vil derfor gerne forsøge, at manipulere vores datasæt så det kun indeholder starttidspunkter der er før dags dato og sluttidspunkter efter den første startsdato i vores DataFrame.

In [ ]:
# 1.8: Hvad er den første dato i vores DataFrame?


In [ ]:
# 1.9: Filtrer trips, så startdatoer efter dags dato og slutdatoer før 1970-01-01 er filtreret væk.
trips = trips[
    (___ < '___') &
    (___ > '___')
]

In [ ]:
# 1.10: Hvad er min og max duration efter vi har filtreret vores dataframe?
print(___)
print(___)

Som det kan ses har vi stadig trip_durations der er udenfor det vi ville forvente. Derfor er vi nødt at lave flere manipulationer af dataframen. Vi kan se på spredningen af vores trip_duration ved at udregne kvantiler. Her vil vi især gerne se kvantilerne 0.01 og 0.99:

In [ ]:
# 1.11: Udregn kvantilerne 0.01, 0.25, 0.5, 0.75 og 0.99 i vores kolonne 'trip_duration'
___

Vi vil gerne have filtreret vores dataframe så vi kun har `trip_duration` der er mellem 0.01 kvantil og 0.99 kvantil. Vi kan derfor filtrere på vores DataFrame igen.

In [ ]:
# 1.12: Filtrer vores trips DataFrame, så vi kun har data mellem 0.01 og 0.99 kvantiler på trips_duration:
# Vi kan filtrere vores kolonner som vi gjorde i 1.9. Her vil vi blot filtrere på trips['trip_duration'].quantile()
trips = trips[
    (___ > ___.quantile(___)) &
    (___ < ___.quantile(___))
]

In [ ]:
# 1.13: Tjek at trip_duration max og min er nogenlunde fornuftige nu:
print("Min. Duration is: ", ___)
print("Max duration is: ", ___)

Efter vi har klargjort vores `trips` DataFrame vil vi gerne have vores `trips` DataFrame og vores `country` DataFrame slået sammen. Dette kan vi gøre vha. funktionen `pd.merge()` funktionen. Vi har dog brug for, at se hvilke kolonner vores `country` DataFrame indeholder.

In [ ]:
# 1.14: Hvilke kolonner indeholder vores country dataframe?
___

Her kan vi se at kolonnen `alpha_2` er den samme som vores kolonne `country_code` i trips dataframen. Vi vil derfor gerne omdøbe kolonnen `alpha_2` i vores country dataframe. Dette kan vi gøre vha. funktionen `df.rename()` hvor vi gerne vil gøre det inplace.

In [ ]:
# 1.15: Omdøb kolonnen alpha_2 til country_code i country dataframen:
country.rename(columns={'___':'___'}, inplace=True)

In [ ]:
# 1.16: Brug pd.merge til at merge de to dataframes på country_code:
merged = pd.merge(___, ___, on='___')

In [ ]:
# 1.17: Hvordan ser de første 5 rækker ud af vores nye DataFrame?


## 2 - Visualiseringer og Dataforståelse

In [ ]:
# 2.1: Brug groupby til at give et overblik over gennemsnittet af trip_duration i de forskellige regioner:


In [ ]:
# 2.2: Brug groupby til at give et overblik over medianen af trip_duration i de forskellige regioner: 


In [ ]:
# 2.3: Brug groupby til at give et overblik over standardafvigelsen af trip_duration i de forskellige regioner: 


In [ ]:
# 2.4: Lav et FacetGrid med boxplots der visualiserer spredningen af trip_duration i de forskellige regioner:
g = sns.FacetGrid(___, col='___', col_wrap=3, sharex=False)
g.map(sns.___, '___')

Som det kan ses i de ovenstående boxplots, har vi stadig en del outliers. Dette er ikke en fejl, blot sådan vores data ser ud. Dette er også forventeligt ud fra hvordan gennemsnittet, medianen og standardafvigelsen er. Vi vil dog gerne prøve at se hvordan fordelingen er i interquartile range. For at gøre dette kan vi filtrere vores data ud fra vores viden om boxplots og outliers. Vi prøver at, lave et FacetGrid igen med merged DataFramen, men hvor vi filtrerer alle rækker med en trip_duration over 100 væk.

In [ ]:
# 2.5: Lav et FacetGrid med boxplots der visualiserer spredningen af trip_duration i de forskellige regioner:
# Brug merged[merged.trip_duration < 100] som data
g = sns.FacetGrid(___, col='___', col_wrap=3, sharex=False)
g.map(sns.___, '___')

In [ ]:
# 2.6: Hvor mange lande er der i vores merged DataFrame? (Husk funktionen .nunique())


In [ ]:
# 2.7: Hvilket land har modtaget flest besøgende?


In [ ]:
# 2.8: Hvilken region har modtaget færrest besøgende?
